# Unified Memory 
### 통합 메모리(Unified Memory)란?
Apple silicon은 Unified Memory 아키텍처를 사용합니다.
애플의 M 시리즈의 통합 메모리(Unified Memory)는 CPU와 GPU가 각각의 별도 메모리를 사용하는 대신 동일한 메모리 뱅크를 사용하는 것을 의미합니다.
지금은 MLX를 배우는 것이 목표이니, 아래 사진만 머릿속에 넣고 지나가겠습니다. 
![UMA](../images/UAM.png)

이미지 출처: https://iboysoft.com/wiki/unified-memory-mac.html 

## MLX로 통합메모리와 놀기
여기서부터 본격적으로 MLX의 특징이 부각됩니다.  
우선 아래와 같이 배열을 선언하면, 따로 위치 지정없이 MLX가 알아서 해줍니다.

In [1]:
import mlx.core as mx
a = mx.random.normal((100,))
b = mx.random.normal((100,))

이제 `a`와 `b` 는 통합 메모리로 넘어갔습니다.

MLX에서는 배열을 특정 장치로 이동시키는 대신, 연산을 수행할 때 장치를 지정합니다. 어떤 장치든 a와 b에 대해 메모리 위치를 옮기지 않고도 모든 연산을 수행할 수 있습니다. 예를 들어:

In [2]:
mx.add(a, b, stream=mx.cpu)
mx.add(a, b, stream=mx.gpu)

array([2.42752, -0.993882, -1.64494, ..., -0.482275, -2.04055, -1.45292], dtype=float32)

위의 예에서는 CPU와 GPU 모두 동일한 덧셈 연산을 수행합니다. 연산들 사이에 의존성이 없으므로, 이 연산들은 병렬로 실행될 수 있으며, 실제로 그렇게 실행될 가능성이 높습니다. MLX에서 스트림의 의미에 대한 자세한 정보는 "Using Streams"를 참조하세요.

위의 덧셈 예제에서는 연산들 간의 의존성이 없기 때문에 경쟁 상태(race condition)의 가능성이 없습니다. 만약 의존성이 존재할 경우, MLX 스케줄러가 이를 자동으로 관리합니다. 예를 들어:

In [3]:
c = mx.add(a, b, stream=mx.cpu)
d = mx.add(a, c, stream=mx.gpu)

이 예에서 두 번째 덧셈 연산은 GPU에서 실행되지만, CPU에서 실행 중인 첫 번째 덧셈 연산의 결과(c)에 의존합니다. MLX는 자동으로 두 스트림 간에 의존성을 삽입하여 첫 번째 연산이 완료되고 c가 사용 가능해진 후에 두 번째 연산이 실행되도록 합니다.

### 활용 예시

MLX가 CPU, GPU에 대해 유연하고 편리한 기능을 제공하는 것까지 보았습니다. 

In [4]:
def fun(a, b, d1, d2):
  x = mx.matmul(a, b, stream=d1)
  for _ in range(500):
      b = mx.exp(b, stream=d2)
  return x, b


In [6]:
a = mx.random.uniform(shape=(4096, 512))
b = mx.random.uniform(shape=(512, 4))

첫 번째 행렬 곱셈(matmul) 연산은 계산 밀도가 높기 때문에 GPU에 적합합니다. 두 번째 연산 시퀀스는 크기가 작아서 GPU에서 오버헤드가 더 클 수 있으므로 CPU에 더 적합합니다.

이 계산을 GPU에서 전부 실행했을 때는 2.8밀리초가 소요됩니다. 하지만 d1=mx.gpu와 d2=mx.cpu로 설정하여 실행하면 1.4밀리초, 즉 약 두 배 더 빠른 시간을 얻을 수 있습니다. 이 시간은 M1 Max에서 측정된 것입니다.

아래의 코드를 실행해보면 이번 챕터를 마무리합시다!


In [19]:
import time
start = time.time()
x, y = fun(a, b, mx.gpu, mx.gpu)
print("Elapsed time: ", (time.time() - start) * 1000)

Elapsed time:  0.8568763732910156


In [21]:
import time
start = time.time()
x, y = fun(a, b, mx.gpu, mx.cpu)
print("Elapsed time: ", (time.time() - start) * 1000)

Elapsed time:  0.6468296051025391


참조: https://macpaw.com/how-to/unified-memory-mac

https://ml-explore.github.io/mlx/build/html/usage/unified_memory.html